<a href="https://colab.research.google.com/github/Aayush360/data_science/blob/master/Bayes_Classifier_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Notebook Imports

In [0]:
import numpy as np
import pandas as pd

### Constants

In [0]:
TRAIN_DATA_PATH ='/content/SpamData/02_Training/train-data.txt'
TEST_DATA_PATH ='/content/SpamData/02_Training/test-data.txt'

TOKEN_HAM_PROB_FILE ='/content/SpamData/03_Test/prob-ham.txt'
TOKEN_SPAM_PROB_FILE ='/content/SpamData/03_Test/prob-spam.txt'
TOKEN_ALL_PROB_FILE = '/content/SpamData/03_Test/prob-all.txt'

TEST_FEATURE_MATRIX = '/content/SpamData/03_Test/test-features.txt'
TEST_TARGET_FILE ='/content/SpamData/03_Test/test-target.txt'



VOCAB_SIZE = 2500

### Read and load features from .txt file into numpy array

In [0]:
sparse_train_data = np.loadtxt(TRAIN_DATA_PATH, delimiter=' ', dtype=int)

In [0]:
sparse_test_data = np.loadtxt(TEST_DATA_PATH, delimiter=' ', dtype=int)

In [0]:
# looking at the first 5 rows

sparse_train_data[:5]

array([[ 0,  0,  1,  2],
       [ 0,  7,  1,  1],
       [ 0, 15,  1,  1],
       [ 0, 18,  1,  1],
       [ 0, 24,  1,  1]])

In [0]:
sparse_train_data[-5:]

array([[5795, 1539,    0,    2],
       [5795, 1903,    0,    1],
       [5795, 1915,    0,    2],
       [5795, 2127,    0,    1],
       [5795, 2279,    0,    1]])

In [0]:
# printing out number of rows in training and test file

print('no of rows in training file is: ', sparse_train_data.shape[0])
print('no of rows in test file is: ', sparse_test_data.shape[0])

no of rows in training file is:  265427
no of rows in test file is:  110522


In [0]:
print('number of unique emails is: ', np.unique(sparse_train_data[:,0]).size) 

number of unique emails is:  4015


In [0]:
 print('number of unique emails in the test file', np.unique(sparse_test_data[:,0]).size)
  
  

number of unique emails in the test file 1724


### Create an empty DataFrame

In [0]:
column_name =['DOC_ID']+['CATEGORY']+list(range(0,VOCAB_SIZE))

In [0]:
column_name[0:5]

['DOC_ID', 'CATEGORY', 0, 1, 2]

In [0]:
len(column_name)

2502

In [0]:
index_names= np.unique(sparse_train_data[:,0])
index_names

array([   0,    1,    2, ..., 5791, 5794, 5795])

In [0]:
full_train_data = pd.DataFrame(index=index_names, columns= column_name)

In [0]:
full_train_data.fillna(value=0, inplace=True)
full_train_data.head()

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,...,2460,2461,2462,2463,2464,2465,2466,2467,2468,2469,2470,2471,2472,2473,2474,2475,2476,2477,2478,2479,2480,2481,2482,2483,2484,2485,2486,2487,2488,2489,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Create a Full Matrix from a Sparse Matrix

In [0]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx=0, word_idx=1, cat_idx=2, freq_idx=3):
  '''
  form a full matrix from a given sparse matrix. Returns a Pandas Dataframe.
  
  Keyword arguments:
  ------------------
  sparse-matrix -- numpy array
  nr_words -- size of the vocabulary. Total number of tokens
  doc_idx -- position of document id index in the sparse matrix. Deafult is 0 (First position in the columns).
  word_idx -- position of word id index in the sparse matrix. Default is 1 (second column)
  cat_idx -- position of category in the sparse matrix spam is 1 and non-spam is 0. Default is 2
  freq_idx -- position of freqency tracker in the sparse matrix that holds the value of no of occurences of word of particular index.
  '''
  column_name =['DOC_ID']+['CATEGORY']+list(range(0,VOCAB_SIZE))
  doc_id_name = np.unique(sparse_matrix[:,0])

  full_matrix = pd.DataFrame(index=doc_id_name, columns= column_name)
  full_matrix.fillna(value=0, inplace=True)

  for i in range(sparse_matrix.shape[0]): # for all the rows in the sparse matrix
    doc_nr = sparse_matrix[i][doc_idx]
    word_id = sparse_matrix[i][word_idx]
    label = sparse_matrix[i][cat_idx]
    occurences = sparse_matrix[i][freq_idx]

    full_matrix.at[doc_nr,'DOC_ID'] = doc_nr # doc_nr tells to go row by row
    full_matrix.at[doc_nr,'CATEGORY'] = label
    full_matrix.at[doc_nr, word_id]= occurences
  
  full_matrix.set_index('DOC_ID', inplace=True)
  return full_matrix





In [0]:
%%time

full_train_data = make_full_matrix(sparse_train_data, VOCAB_SIZE)

CPU times: user 8.54 s, sys: 111 ms, total: 8.65 s
Wall time: 8.64 s


In [0]:
full_train_data.head()

,CATEGORY,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,2460,2461,2462,2463,2464,2465,2466,2467,2468,2469,2470,2471,2472,2473,2474,2475,2476,2477,2478,2479,2480,2481,2482,2483,2484,2485,2486,2487,2488,2489,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1,2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,2,0,1,2,1,1,0,0,0,0,0,0,0,0,0,0,2,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,2,0,2,0,0,3,0,0,0,0,3,0,0,0,0,0,4,4,2,0,0,0,2,0,0,0,2,0,2,1,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,3,0,0,1,0,1,1,0,0,0,0,0,0,0,0,3,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,8,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,1,2,4,2,3,1,1,1,0,1,1,4,0,1,0,2,0,2,2,4,3,0,0,1,0,0,0,2,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Training the Naive Bayes Model

##### Calculating the probability of Spam



In [0]:
spam_count=0
for cat in full_train_data.CATEGORY:
  if cat == 1:
    spam_count=spam_count+1
  
spam_count

1250

In [0]:
# we can directly get the value of number of spam messages as:

full_train_data.CATEGORY.sum()

1250

In [0]:
prob_spam = spam_count/full_train_data.shape[0]
prob_spam

0.31133250311332505

#### Total number of words or Tokens

In [0]:
total_words = sparse_train_data.shape[0] + sparse_test_data.shape[0]
print('the total number of words in the entire dataset is: ', total_words)

the total number of words in the entire dataset is:  375949


In [0]:
full_train_features = full_train_data.loc[:, full_train_data.columns != 'CATEGORY']
full_train_features

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2460,2461,2462,2463,2464,2465,2466,2467,2468,2469,2470,2471,2472,2473,2474,2475,2476,2477,2478,2479,2480,2481,2482,2483,2484,2485,2486,2487,2488,2489,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,0,1,2,1,1,0,0,0,0,0,0,0,0,0,0,2,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,0,2,0,0,3,0,0,0,0,3,0,0,0,0,0,4,4,2,0,0,0,2,0,0,0,2,0,2,1,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,0,0,1,0,1,1,0,0,0,0,0,0,0,0,3,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,8,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,2,4,2,3,1,1,1,0,1,1,4,0,1,0,2,0,2,2,4,3,0,0,1,0,0,0,2,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5789,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5790,2,2,1,0,0,0,1,0,1,0,0,1,1,3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5791,2,0,1,1,0,1,2,0,1,2,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
# summing up everything in the entire cells gives us total number of tokens per emails


emails_lengths = full_train_features.sum(axis=1)
emails_lengths.shape


(4015,)

In [0]:
emails_lengths[:5]

DOC_ID
0     50
1     76
2     87
3     76
4    136
dtype: int64

In [0]:
# to get the sum of entire token sum up the values in the entire series

total_wc = emails_lengths.sum()
total_wc

445891

In [0]:
# total number of words in spam emails


spam_data_set_temp = full_train_data.loc[full_train_data['CATEGORY'].values == 1,:]
spam_data_set = spam_data_set_temp.loc[:, spam_data_set_temp.columns !='CATEGORY']

spam_email_length = spam_data_set.sum(axis=1)

print('spam email length is: ', spam_email_length.shape)

spam_wc = spam_email_length.sum()
print('total number of word counts in spam email is:' ,spam_wc)

print('average length of spam email is : ', spam_wc/spam_email_length.shape[0])

spam email length is:  (1250,)
total number of word counts in spam email is: 195645
average length of spam email is :  156.516


In [0]:
# total number of words in non-spam emails


ham_data_set_temp = full_train_data.loc[full_train_data['CATEGORY'].values == 0,:]
ham_data_set = ham_data_set_temp.loc[:, ham_data_set_temp.columns !='CATEGORY']

ham_email_length = ham_data_set.sum(axis=1)
print('ham email length is: ', ham_email_length.shape)

ham_wc = ham_email_length.sum()
ham_wc
print('total number of word counts in spam email is: ', ham_wc)

print('average length of ham email is :{:.0f}'.format(ham_wc/ham_email_length.shape[0]))# rounding nearest whole number

ham email length is:  (2765,)
total number of word counts in spam email is:  250246
average length of ham email is :91


In [0]:
# alternate approach

spam_lengths = emails_lengths[full_train_data.CATEGORY ==1] # subsetting based on the condition
spam_lengths.shape

(1250,)

In [0]:
spam_wcs = spam_lengths.sum()
spam_wcs

195645

In [0]:
# check if you have subsetted things correctly

emails_lengths.shape[0] - spam_email_length.shape[0] - ham_email_length.shape[0]

0

In [0]:
total_wc == spam_wc + ham_wc

True

In [0]:
# is spam or non-spam email tend to be longer on average?? 

### Summing the tokens occuring in Spam

In [0]:
full_train_features.shape

(4015, 2500)

In [0]:
# create a subset of full_train_features dataset that contains spam and non-spam emails separately

train_spam_tokens = full_train_features.loc[full_train_data.CATEGORY ==1]
train_spam_tokens.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2460,2461,2462,2463,2464,2465,2466,2467,2468,2469,2470,2471,2472,2473,2474,2475,2476,2477,2478,2479,2480,2481,2482,2483,2484,2485,2486,2487,2488,2489,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,0,1,2,1,1,0,0,0,0,0,0,0,0,0,0,2,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,0,2,0,0,3,0,0,0,0,3,0,0,0,0,0,4,4,2,0,0,0,2,0,0,0,2,0,2,1,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,0,0,1,0,1,1,0,0,0,0,0,0,0,0,3,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,8,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,2,4,2,3,1,1,1,0,1,1,4,0,1,0,2,0,2,2,4,3,0,0,1,0,0,0,2,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
train_spam_tokens.shape #no of spam_emails x no of vocab_words

(1250, 2500)

In [0]:
# sum up all the tokens columns by columns i.e for frequency of tokens in spam messages

summed_spam_tokens = train_spam_tokens.sum(axis=0) +1 # adding 1 to make the calculation non-zero, some tokens may not appear

# this method of making non-zero is called Laplace Smoothing 

summed_spam_tokens.shape

(2500,)

In [0]:
summed_spam_tokens.tail()

2495    19
2496     2
2497    23
2498    10
2499     4
dtype: int64

### Summing the tokens occuring in Ham

In [0]:
train_ham_tokens = full_train_features.loc[full_train_data.CATEGORY ==0]

summed_ham_tokens = train_ham_tokens.sum(axis=0) +1 # adding 1 to make the calculation non-zero, some tokens may not appear
summed_ham_tokens.shape # no of vocabs

(2500,)

In [0]:
summed_ham_tokens.tail()

2495    13
2496    19
2497     1
2498    16
2499    35
dtype: int64

In [0]:
train_ham_tokens[2499].sum(axis=0)+1

35

### P(Token|Spam) - Probability that a Token occurs given that the emails is Spam

In [0]:
prob_token_spam = summed_spam_tokens/(spam_wc+VOCAB_SIZE) # adding 2500 to deno as well, since we've added 1 to every words in summed_spam_tokens 
# this is the part of smoothing
# dividing every word count in spam by total word_count in spam 

In [0]:
prob_token_spam[:5]

0    0.009049
1    0.004885
2    0.006838
3    0.010593
4    0.006803
dtype: float64

In [0]:
prob_token_spam.sum()

1.0

### P(Token|Ham) - Probability that the Token occurs given that the email is Non-Spam

In [0]:
prob_token_ham = summed_ham_tokens/ (ham_wc +VOCAB_SIZE) 
prob_token_ham[:5]

0    0.020863
1    0.009887
2    0.008063
3    0.003735
4    0.006311
dtype: float64

In [0]:
prob_token_ham.sum()

1.0

### Probability that a token occurs in the Entire Email

In [0]:
summed_tokens = full_train_features.sum(axis=0) 

# no need Laplace Smoothing here as we will not end up with 0, coz we have taken 2500 most frequent words

summed_tokens[:5]

0    7064
1    3465
2    3391
3    3041
4    2941
dtype: int64

In [0]:
prob_token = summed_tokens/ (total_wc )
prob_token.sum() 

1.0

In [0]:
prob_token[:5]

0    0.015842
1    0.007771
2    0.007605
3    0.006820
4    0.006596
dtype: float64

### Save the trained Model


In [0]:
np.savetxt(TOKEN_SPAM_PROB_FILE,prob_token_spam)
np.savetxt(TOKEN_HAM_PROB_FILE, prob_token_ham)
np.savetxt(TOKEN_ALL_PROB_FILE, prob_token)

### Prepare Test Data


In [0]:
%%time

full_test_data = make_full_matrix(sparse_test_data, VOCAB_SIZE)

CPU times: user 3.97 s, sys: 9.49 ms, total: 3.98 s
Wall time: 4 s


In [0]:
full_test_data.head()

,CATEGORY,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,2460,2461,2462,2463,2464,2465,2466,2467,2468,2469,2470,2471,2472,2473,2474,2475,2476,2477,2478,2479,2480,2481,2482,2483,2484,2485,2486,2487,2488,2489,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
8,1,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12,1,0,2,1,1,0,0,0,1,2,0,0,0,0,0,0,1,1,0,1,1,2,1,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
15,1,3,0,1,0,0,0,1,0,0,1,0,1,0,0,1,0,2,0,1,2,0,0,0,0,2,0,1,0,0,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
17,1,5,1,2,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
19,1,1,2,0,0,0,0,1,1,0,0,0,1,0,2,0,0,1,0,1,0,5,0,0,1,0,0,0,1,1,1,1,3,0,0,0,0,1,0,2,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
X_test = full_test_data.loc[:, full_test_data.columns != 'CATEGORY']
y_test = full_test_data.CATEGORY

In [0]:
np.savetxt(TEST_FEATURE_MATRIX, X_test)
np.savetxt(TEST_TARGET_FILE, y_test)